In [7]:
# Settings
#filename    = '/home/axel/PycharmProjects/pax/tpc_kr_150410_8k.hdf5'
filename     = '/home/axel/ownCloud/documents/Masterarbeit/TPC/rawdata/2015_11_02/Kr83m_valve_50min_open/Kr83m_gas_cold_151102_1k.root'
#filename    = '/home/axel/ownCloud/documents/Masterarbeit/TPC/rawdata/2015_10_02/Kr_Measurement_1000TH_1_151002.hdf5'
#filename    = '/media/axel/cf2b7088-fe0f-419a-a101-07493d88971c/kabelchange.hdf5.hdf5'
file_format = 'root'
#loglevel    = 'DEBUG'

In [5]:
import logging
log = logging.getLogger('Gaseous basic statistics')
log.setLevel('DEBUG')

import numpy as np

import matplotlib   # Needed for font size spec, color map transformation function bla bla
%matplotlib inline
matplotlib.rc('font', size=16)
import matplotlib.pyplot as plt

import matplotlib.dates as md
import datetime as dt
import time

# Optional progress bar
try:
    from tqdm import tqdm
except ImportError:
    log.debug("You don't have tqdm, I can't give you a nice progress bar...")
    def dummy(*args,**kwargs):
        return args[0]
    tqdm = dummy

from recarray_tools import append_fields, fields_view, group_by, filter_on_fields, fields_data

peak_types = [b's1', b's2', b'unknown', b'noise', b'lone_hit']

In [11]:
##
# Load the data
##
try:
    events = None
    hits = None
    peaks = None
except Exception:
    pass

log.debug("Now loading %s (file format=%s)..." % (filename, file_format))

# Slurp peaks and events into memory
# WARNING: For a large dataset, use pax to remove big low-level fields first
# (area_per_channel, does_channel_contribute, does_channel_have_noise)
try:
    # Load the file using pax 3's IO code
    from pax.formats import flat_data_formats
    ioformat = flat_data_formats[file_format]()
    ioformat.open(filename, 'r')
    events = ioformat.read_data('tree')
    try:
        peaks = ioformat.read_data('Peak')
    except KeyError:
        peaks = None
        log.info("There is no peak table")
    hits = ioformat.read_data('hits')
    ioformat.close()
except ImportError:
    log.debug("You don't have pax 3 installed, falling back to HDF5-specific code...")
    import h5py
    f = h5py.File(filename)
    events = f.get('Event')[:]
    peaks = f.get('Peak')[:]
    hits = f.get('hits')[:]
    f.close()
#if peaks != None:
    log.info("Loaded %s, found %d hits (%0.2f MB RAM), clustered into %d peaks (%0.2f MB RAM) in %d events (%0.2f MB RAM)" % (
    filename, len(hits), hits.nbytes/10**6, len(peaks), peaks.nbytes/10**6, len(events), events.nbytes/10**6))
#if len(events) == 0:
#    raise ValueError("You don't have any events in this dataset!")

DEBUG:Gaseous basic statistics:Now loading /home/axel/ownCloud/documents/Masterarbeit/TPC/rawdata/2015_11_02/Kr83m_valve_50min_open/Kr83m_gas_cold_151102_1k.root (file format=root)...


ValueError: Strange branch events in tree tree?

In [ ]:
log.info("Loaded %s, found %d hits (%0.2f MB RAM), clustered into %d peaks (%0.2f MB RAM) in %d events (%0.2f MB RAM)" % (
    filename, len(hits), hits.nbytes/10**6, len(peaks), peaks.nbytes/10**6, len(events), events.nbytes/10**6))

In [ ]:
# Adding extra fields
log.info("Loaded %d events (%0.2f MB RAM)" %(len(events), events.nbytes/1e6))
fig, ax = plt.subplots(1,1, figsize=(12,8))
ax.set_ylabel("rate [events / s]")
ax.set_xlabel("time [s]")
ax.set_title("event rate after opening valve to Kr83m source")
a = ax.hist((events["start_time"]-events["start_time"][0])//1e9, bins=(events["start_time"][-1]-events["start_time"][0])//1e9 , histtype="step")

In [ ]:
log.info("Found %d hits (%s MB RAM)"% (len(hits),hits.nbytes//1e6))

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(16,20), squeeze=False)
area_right_boundary = 200
area_bins = 100

if True:
    
    ### making histogram of hits
    z = np.histogram(hits["channel"], bins=14, range=(0,14))
    print(z)
    plt.cla()
    ax1.bar(range(14),z[0], align="center")
    ax1.set_xlabel("channel number")
    ax1.set_ylabel("number of hits")
    ax1.set_title("total contribution of individual channels")
    ax1.set_xlim((-1,14))
    
    ### now checking how peaks were identified
    peak_type_sums = dict()
    for peaktype in peak_types:
        z = (peaks["type"] == peaktype)
        peak_type_sums[peaktype] = sum(z)
    ax2.bar(range(len(peak_types)), peak_type_sums.values(),align="center")
    ax2.set_ylabel("number of peaks")
    xlabels = [""]
    xlabels.extend(list(peak_type_sums.keys()))
    ax2.set_xticklabels(labels= xlabels, rotation = 30)
    ax2.set_title("classification of peaks")
    ax2.set_yscale("log")
    #ax2.set_xlim((-1,14))
    
    ### 
    import pandas as pd
    # making a histogram of contributing channels towards peaks
    # looking at coincidential peaks (type != lone_hit) 
    s1_peaks = peaks["type"] == b"s1"
    lone_hits = peaks["type"] == b"lone_hit"

    a = pd.DataFrame(peaks[s1_peaks]["hits_per_channel"])
    b = pd.DataFrame(peaks[lone_hits]["hits_per_channel"])
    ax3.bar(list(a), list(a.sum()),align="center")
    ax3.set_title("contribution to  's1' peaks")
    
    ax4.set_title("contribution to 'lone hits' peaks")
    ax4.bar(list(b), list(b.sum()),align="center")
    
    ax3.set_xlim((-1,14))
    ax3.set_ylabel("number of hits")
    ax4.set_xlim((-1,14))
    ax4.set_ylabel("number of hits")
    
    ax5.set_title("peak area")
    ax5.hist(peaks["area"], bins=area_bins, histtype="step", range=(0,area_right_boundary), label="all peaks")
    ax5.hist(peaks[s1_peaks]["area"], bins=area_bins, histtype="step", range=(0,area_right_boundary), label = "s1")
    ax5.set_xlabel("peak area [p.e.]")
    #ax5.set_ylabel("#")
    ax5.set_xlim((0,area_right_boundary))
    ax5.legend(loc="best")
    ax5.set_yscale("log")
    
    ax6.set_title("number of peaks in events")
    ax6.hist(events["n_peaks"], bins=10, histtype="step", range=(0,10), align="left")
    ax6.set_xlabel("n_peaks")
    
    fig.tight_layout()
    fig.show()

In [ ]:
### Now finding events that contain >=2 peaks
### Then plotting energy of first peak (ordered by area) vs the second peak

multi_peak_events = events[events["n_peaks"]>= 2]

peak_by_event = group_by(peaks, "Event")

main_peaks = []
secondary_peaks = []

for event_peaks in peak_by_event:
    #print(event_peaks["area"])
    event_peaks = sorted(event_peaks, key= lambda x : x["area"],reverse=True)
    #print(event_peaks[0]["area"])
    if len(event_peaks)<= 1:
        continue
    if event_peaks[0]["n_contributing_channels"] <= 1:
        continue
    if event_peaks[1]["n_contributing_channels"] <= 0:
        continue
    main_peaks.append(event_peaks[0])
    secondary_peaks.append(event_peaks[1])
    

main_areas = [peak["area"] for peak in main_peaks]
secondary_areas =[peak["area"] for peak in secondary_peaks]

fig, ax1 = plt.subplots(1,1, figsize=(16,10))
plt.hist2d(main_areas, secondary_areas, bins=[120,120], range=((0,200),(0,20)))
ax1.set_xlabel("area of main s1 peak [p.e.]")
ax1.set_ylabel("area of secondary s1 peak [p.e.]")
cbar = plt.colorbar()
cbar.set_label("counts")
#ax.set_xlim(0,150)
#ax.set_ylim(0,50)

#ax2.hist(peaks["index_of_maximum"], bins=512, histtype="step")
#ax2.set_yscale("log")
fig.show()


    
#ax.hist2d?

In [ ]:
sorted([1,2,3,4,8,5], reverse =True)


In [ ]:
import plot_by_channel
print(hits.dtype)
plot_by_channel.make_plot(hits, "height",range=(0,10), xlim=(0,10))

In [ ]:
f = h5py.File(filename)

plt.yscale("log")
plt.hist(f.get("Peak")["height"], bins=100, histtype="step", range=(0,500))
f.close()

In [ ]:
import pax

In [ ]:
pax

In [ ]:
peaks.dtype.names